In [1]:
#!pip install fair-esm torch_geometric

In [2]:
import esm
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from torch_geometric.utils import negative_sampling
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import pickle, os

In [3]:
# --- ESM model ---
esm_model, esm_alphabet = esm.pretrained.esm2_t6_8M_UR50D()  # Lightweight for speed
batch_converter = esm_alphabet.get_batch_converter()
esm_model.eval()

def embed_sequence_esm(seq):
    seq = seq.replace(" ", "").upper()
    batch_labels, batch_strs, batch_tokens = batch_converter([("protein", seq)])
    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[6], return_contacts=False)
    token_reps = results["representations"][6][0, 1:len(seq)+1].mean(0)
    return token_reps.cpu().numpy()

In [4]:
# Load dataset
# -----------------------------
df = pd.read_csv("df_actions_148k.csv")

In [5]:
df.head()

sequence_a  \
0  MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...   
1  MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...   
2  MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...   
3  MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...   
4  MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...   

                                          sequence_b             item_id_a  \
0  MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQ...  9606.ENSP00000000233   
1  MAQAAKQLKKIKDIEAQALQEQKEKEESNRKRRNRSRDRKKKADAA...  9606.ENSP00000000233   
2  MAQAAKQLKKIKDIEAQALQEQKEKEESNRKRRNRSRDRKKKADAA...  9606.ENSP00000000233   
3  MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQ...  9606.ENSP00000000233   
4  MAQAAKQLKKIKDIEAQALQEQKEKEESNRKRRNRSRDRKKKADAA...  9606.ENSP00000000233   

              item_id_b        mode is_directional a_is_acting  score  
0  9606.ENSP00000263025       ptmod              f           f    150  
1  9606.ENSP00000265709    reaction              f           f    908  
2  9606.ENSP00000265709   catalysis              t           t    908  
3  9606.ENSP00000263025  inhibition              t           t    154  
4  9606.ENSP00000265709     binding              f           t    908

In [6]:
# Map mode to edge type (int)
df['edge_type'] = pd.factorize(df['mode'])[0]
mode_to_int = dict(zip(df['mode'], df['edge_type']))

In [7]:
# --- Unique proteins and sequences ---
proteins_a = df[['item_id_a', 'sequence_a']].rename(columns={'item_id_a': 'item_id', 'sequence_a': 'sequence'})
proteins_b = df[['item_id_b', 'sequence_b']].rename(columns={'item_id_b': 'item_id', 'sequence_b': 'sequence'})
all_proteins = pd.concat([proteins_a, proteins_b]).drop_duplicates("item_id").set_index("item_id")
protein_to_idx = {pid: i for i, pid in enumerate(all_proteins.index)}

In [8]:
# Check sample sequences before embedding
print("\n🔍 Sample sequences:")
print(all_proteins["sequence"].dropna().head(5))

# Redefine embed_sequence_esm with logging
def embed_sequence_esm(seq):
    try:
        seq = seq.replace(" ", "").upper()
        if not seq or not isinstance(seq, str) or len(seq) < 5:
            print("❌ Invalid sequence:", seq)
            return None

        batch_labels, batch_strs, batch_tokens = batch_converter([("protein", seq)])
        with torch.no_grad():
            results = esm_model(batch_tokens, repr_layers=[6], return_contacts=False)
        token_reps = results["representations"][6][0, 1:len(seq)+1]
        return token_reps.mean(0).cpu().numpy()
    except Exception as e:
        print(f"❌ Failed to embed sequence: {e}")
        return None

# Embedding loop with logging
protein_embeddings = {}
for pid in tqdm(all_proteins.index, desc="Embedding proteins"):
    try:
        seq = all_proteins.loc[pid, "sequence"]
        if not isinstance(seq, str) or len(seq.strip()) == 0:
            print(f"⚠️ Skipping {pid}: Empty or invalid sequence")
            continue
        embedding = embed_sequence_esm(seq)
        if embedding is not None:
            protein_embeddings[pid] = embedding
            print(f"✅ Embedded {pid}")
        else:
            print(f"⚠️ Skipped {pid}: Embedding returned None")
    except Exception as e:
        print(f"❌ Error embedding {pid}: {e}")

print(f"\n✅ Total embeddings generated: {len(protein_embeddings)}")



🔍 Sample sequences:
item_id
9606.ENSP00000000233    MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...
9606.ENSP00000000412    MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
9606.ENSP00000000442    MSSQVVGIEPLYIKAEPASPDSPKGSSETETEPPVALAPGPAPTRC...
9606.ENSP00000001008    MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
9606.ENSP00000001146    MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
Name: sequence, dtype: object


Embedding proteins:   0%|                                                             | 2/5189 [00:00<05:57, 14.50it/s]

✅ Embedded 9606.ENSP00000000233
✅ Embedded 9606.ENSP00000000412


Embedding proteins:   0%|                                                             | 4/5189 [00:00<09:02,  9.56it/s]

✅ Embedded 9606.ENSP00000000442
✅ Embedded 9606.ENSP00000001008


...

✅ Embedded 9606.ENSP00000297029
✅ Embedded 9606.ENSP00000297316
✅ Embedded 9606.ENSP00000314132


Embedding proteins: 100%|█████████████████████████████████████████████████████████▉| 5188/5189 [28:14<00:00,  6.12it/s]

✅ Embedded 9606.ENSP00000314606


Embedding proteins: 100%|██████████████████████████████████████████████████████████| 5189/5189 [28:14<00:00,  3.06it/s]

✅ Embedded 9606.ENSP00000296280

✅ Total embeddings generated: 5189


In [9]:
# cache_path = "esm2_embeddings_linkprediction_rgcn.pkl"

# if os.path.exists(cache_path):
#     with open(cache_path, "rb") as f:
#         protein_embeddings = pickle.load(f)
# else:
#     protein_embeddings = {}
#     for pid in tqdm(all_proteins.index, desc="Embedding proteins"):
#         try:
#             seq = all_proteins.loc[pid, "sequence"]
#             if not isinstance(seq, str) or len(seq.strip()) == 0:
#                 print(f"⚠️ Skipping {pid}: Empty or invalid sequence")
#                 continue
#             embedding = embed_sequence_esm(seq)
#             protein_embeddings[pid] = embedding
#         except Exception as e:
#             print(f"❌ Error embedding {pid}: {e}")
#     with open(cache_path, "wb") as f:
#         pickle.dump(protein_embeddings, f)

# print(f"✅ Total embeddings generated: {len(protein_embeddings)}")

In [10]:
# --- Node feature matrix ---
if len(protein_embeddings) == 0:
    raise ValueError("❌ No embeddings were generated. Check your embedding function or input sequences.")

embedding_dim = len(next(iter(protein_embeddings.values())))
x = np.zeros((len(protein_to_idx), embedding_dim), dtype=np.float32)
for pid, idx in protein_to_idx.items():
    x[idx] = protein_embeddings[pid]
x = torch.tensor(x, dtype=torch.float)

# --- Direction-aware edge construction ---
src_nodes = []
dst_nodes = []
edge_types = []

for _, row in df.iterrows():
    a = row['item_id_a']
    b = row['item_id_b']
    edge_label = mode_to_int[row['mode']]
    
    # Handle direction
    if row['is_directional'] == 't':
        if row['a_is_acting'] == 't':
            src_nodes.append(protein_to_idx[a])
            dst_nodes.append(protein_to_idx[b])
        else:
            src_nodes.append(protein_to_idx[b])
            dst_nodes.append(protein_to_idx[a])
        edge_types.append(edge_label)
    else:
        # Undirected — add both directions
        src_nodes.append(protein_to_idx[a])
        dst_nodes.append(protein_to_idx[b])
        edge_types.append(edge_label)

        src_nodes.append(protein_to_idx[b])
        dst_nodes.append(protein_to_idx[a])
        edge_types.append(edge_label)

# Convert to tensors
edge_index = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)
edge_type = torch.tensor(edge_types, dtype=torch.long)

# --- PyG data object ---
data = Data(x=x, edge_index=edge_index, edge_type=edge_type)

In [11]:
# --- Train/val/test split ---
ei = data.edge_index.numpy()
et = data.edge_type.numpy()
ei_train, ei_test, et_train, et_test = train_test_split(ei.T, et, test_size=0.2, random_state=42)
ei_train, ei_val, et_train, et_val = train_test_split(ei_train, et_train, test_size=0.1, random_state=42)

ei_train = torch.tensor(ei_train, dtype=torch.long).t()
ei_val = torch.tensor(ei_val, dtype=torch.long).t()
ei_test = torch.tensor(ei_test, dtype=torch.long).t()

et_train = torch.tensor(et_train, dtype=torch.long)
et_val = torch.tensor(et_val, dtype=torch.long)
et_test = torch.tensor(et_test, dtype=torch.long)

neg_train = negative_sampling(ei_train, num_nodes=data.num_nodes)
neg_val = negative_sampling(ei_val, num_nodes=data.num_nodes)
neg_test = negative_sampling(ei_test, num_nodes=data.num_nodes)

In [12]:
# --- RGCN model ---
class RGCNLinkPredictor(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_relations):
        super().__init__()
        self.conv1 = RGCNConv(in_dim, hidden_dim, num_relations)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations)

    def encode(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type))
        return self.conv2(x, edge_index, edge_type)

    def decode(self, z, edge_index):
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)

model = RGCNLinkPredictor(data.num_features, hidden_dim=64, out_dim=32, num_relations=len(mode_to_int))
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [13]:
# --- Training ---
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, ei_train, et_train)
    pos_score = model.decode(z, ei_train)
    neg_score = model.decode(z, neg_train)
    pos_labels = torch.ones_like(pos_score)
    neg_labels = torch.zeros_like(neg_score)
    loss = F.binary_cross_entropy_with_logits(pos_score, pos_labels) + \
           F.binary_cross_entropy_with_logits(neg_score, neg_labels)
    loss.backward()
    optimizer.step()
    return loss.item()

In [14]:
# --- Evaluation ---
def evaluate(ei_pos, et_pos, ei_neg):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, ei_pos, et_pos)
        pos_score = model.decode(z, ei_pos).sigmoid().cpu().numpy()
        neg_score = model.decode(z, ei_neg).sigmoid().cpu().numpy()
        y_true = np.concatenate([np.ones_like(pos_score), np.zeros_like(neg_score)])
        y_score = np.concatenate([pos_score, neg_score])
        roc = roc_auc_score(y_true, y_score)
        pr = average_precision_score(y_true, y_score)
        return roc, pr

In [15]:
# --- Train loop ---
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        roc, pr = evaluate(ei_val, et_val, neg_val)
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")

# --- Final test evaluation ---
roc, pr = evaluate(ei_test, et_test, neg_test)
print(f"\n🧪 Test ROC-AUC: {roc:.4f} | Test PR-AUC: {pr:.4f}")

Epoch 010 | Loss: 1.3107 | Val ROC-AUC: 0.8440 | PR-AUC: 0.8713
Epoch 020 | Loss: 1.1550 | Val ROC-AUC: 0.8381 | PR-AUC: 0.8764
Epoch 030 | Loss: 1.0866 | Val ROC-AUC: 0.8351 | PR-AUC: 0.8797
Epoch 040 | Loss: 1.0202 | Val ROC-AUC: 0.8504 | PR-AUC: 0.8904
Epoch 050 | Loss: 0.9814 | Val ROC-AUC: 0.8445 | PR-AUC: 0.8865
Epoch 060 | Loss: 0.9646 | Val ROC-AUC: 0.8525 | PR-AUC: 0.8917
Epoch 070 | Loss: 0.9488 | Val ROC-AUC: 0.8521 | PR-AUC: 0.8917
Epoch 080 | Loss: 0.9358 | Val ROC-AUC: 0.8522 | PR-AUC: 0.8925
Epoch 090 | Loss: 0.9238 | Val ROC-AUC: 0.8537 | PR-AUC: 0.8943
Epoch 100 | Loss: 0.9164 | Val ROC-AUC: 0.8476 | PR-AUC: 0.8911
Epoch 110 | Loss: 0.9079 | Val ROC-AUC: 0.8513 | PR-AUC: 0.8941
Epoch 120 | Loss: 0.9018 | Val ROC-AUC: 0.8465 | PR-AUC: 0.8915
Epoch 130 | Loss: 0.8962 | Val ROC-AUC: 0.8475 | PR-AUC: 0.8923
Epoch 140 | Loss: 0.8912 | Val ROC-AUC: 0.8463 | PR-AUC: 0.8917
Epoch 150 | Loss: 0.8869 | Val ROC-AUC: 0.8444 | PR-AUC: 0.8907
Epoch 160 | Loss: 0.8834 | Val ROC-AUC: 

In [16]:
# Save model
torch.save(model.state_dict(), "rgcn_esm2_linkpredictor_directional_148k.pth")